In [ ]:
import altair as alt
import pandas as pd

- https://altair-viz.github.io/gallery/line_chart_with_datum.html
- https://altair-viz.github.io/gallery/falkensee.html
- https://www.radix-ui.com/docs/colors/palette-composition/the-scales

In [ ]:
alt.renderers.set_embed_options(renderer="svg")

In [ ]:
DATA: str = "https://raw.githubusercontent.com/dssgPT/Plotting-Good-DSSG/main/desafios/002_Qualidade_do_Ar_em_Portugal_EEA/qualidade_do_ar.csv"

In [ ]:
df = pd.read_csv(DATA, index_col=0)
df["cidade"] = df["cidade"].str.strip()

In [ ]:
df["cidade"].unique()

In [21]:
cols = [
    "cidade",
    "poluente",
    "unidade_de_nivel_de_poluicao",
    "metrica",
    "nome_estacao",
    "ano",
    "nível_de_poluicao",
]

poluente = "O3"
poluente_domain = [0, 80]

# cidade = "Lisboa (greater city)"
cidade = "Porto (greater city)"

# https://www.lisboa.pt/municipio/camara-municipal/identidade-grafica
# city_color = "#000000"
# https://www.cm-porto.pt/marca-porto/marca-porto
city_color = "rgb(15,47,127)"

df_cities = df.query(
    "poluente == @poluente & metrica == 'Média anual' & cidade == @cidade"
).sort_values(by=["ano", "nome_estacao"])[cols]

In [22]:
df_cities.head()

,cidade,poluente,unidade_de_nivel_de_poluicao,metrica,nome_estacao,ano,nível_de_poluicao
2631,Porto (greater city),O3,ug/m3,Média anual,Custóias-Matosinhos,1999,37.371
1070,Porto (greater city),O3,ug/m3,Média anual,Ermesinde-Valongo,1999,32.249
3532,Porto (greater city),O3,ug/m3,Média anual,Custóias-Matosinhos,2000,37.434
1341,Porto (greater city),O3,ug/m3,Média anual,Ermesinde-Valongo,2000,35.325
3623,Porto (greater city),O3,ug/m3,Média anual,Leça do Balio-Matosinhos,2000,31.986


In [23]:
df_cities["nome_estacao"].value_counts(dropna=False)

Ermesinde-Valongo            22
Leça do Balio-Matosinhos     19
Custóias-Matosinhos          17
Meco-Perafita                16
Sobreiras-Lordelo do Ouro    14
Avintes                      12
Name: nome_estacao, dtype: int64

In [24]:
max_value = df_cities["nível_de_poluicao"].max()
max_value

63.418

In [25]:
min_value = df_cities["nível_de_poluicao"].min()
min_value

25.095

In [26]:
df_cities[df_cities["nível_de_poluicao"] == max_value]

,cidade,poluente,unidade_de_nivel_de_poluicao,metrica,nome_estacao,ano,nível_de_poluicao
2071,Porto (greater city),O3,ug/m3,Média anual,Ermesinde-Valongo,2021,63.418


In [27]:
df_cities[df_cities["nível_de_poluicao"] == min_value]

,cidade,poluente,unidade_de_nivel_de_poluicao,metrica,nome_estacao,ano,nível_de_poluicao
1306,Porto (greater city),O3,ug/m3,Média anual,Leça do Balio-Matosinhos,2013,25.095


In [ ]:
# df_cities["ano"] = pd.to_datetime(df_cities["ano"], format="%Y")

In [ ]:
FONT: str = "Rubik"

In [ ]:
years = range(df_cities["ano"].min(), df_cities["ano"].max() + 1)

df_bg = pd.DataFrame([{"ano": year} for year in years])

df_bg.head()

In [ ]:
base = alt.Chart(df_cities)

ticks = base.mark_tick(color=city_color, opacity=1).encode(
    x=alt.X(
        "ano:O",
        axis=alt.Axis(
            title=None,
            labelExpr="substring(datum.label, 2, 4)",
            labelAngle=0,
            labelAlign="center",
        ),
        scale=alt.Scale(domain=list(range(df["ano"].min(), df["ano"].max() + 1))),
    ),
    y=alt.Y(
        "nível_de_poluicao:Q",
        scale=alt.Scale(domain=poluente_domain),
        axis=alt.Axis(title=None, domain=False),
    ),
)

max_line = base.mark_rule(color="red").encode(y=alt.datum(max_value))
min_line = base.mark_rule(color="red").encode(y=alt.datum(min_value))

bg = (
    alt.Chart(df_bg)
    .mark_rect()
    .encode(
        x=alt.X("ano:O"),
        color=alt.Color(
            "ano:N", legend=None, scale=alt.Scale(range=["lightgray", "white"])
        ),
    )
)

(bg + max_line + min_line + ticks).properties(title=cidade).configure_view(
    strokeWidth=0
).configure_axis(labelFont=FONT)

---